### Utiliser la méthode du coude pour déterminer le nombre optimal de clusters.

Chargement de données

In [2]:
import pandas as pd
import os

df_standard = pd.read_csv(os.getcwd() + '/../data/processed/dataset-standard.csv')

Utilisation de la méthode du coude, pour detecter K

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

wcss = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_standard)
    wcss.append(kmeans.inertia_)

# Plot
plt.plot(K_range, wcss, marker='o')
plt.xlabel("Number of clusters K")
plt.ylabel("WCSS (inertia)")
plt.title("Elbow Method")
plt.show()

Utilisation de la méthode de silhouette, pour detecter K

In [ ]:
from sklearn.metrics import silhouette_score

sil_scores = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(df_standard)
    score = silhouette_score(df_standard, labels)
    sil_scores.append(score)

plt.plot(range(2, 11), sil_scores, marker='o')
plt.xlabel("Number of clusters K")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Method")
plt.show()

D’après les résultats des deux méthodes, la valeur optimale de K est 2.

### Entraîner le modèle K-Means avec le k optimal.

Avec K = 2

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)

kmeans.fit(df_standard)

- Ajouter une colonne (Cluster) pour la classification de données

In [ ]:
labels = kmeans.labels_

df_standard['Cluster'] = labels

print(df_standard.head())

Visualisation des résultats en utilisant uniquement deux colonnes : Glucose et BMI. Les clusters ne sont pas clairement séparés, car les données nécessitent 8 dimensions et nous les projetons dans un espace en deux dimensions.

In [ ]:
import matplotlib.pyplot as plt

X = df_standard[['Glucose', 'BMI']].values

plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels_, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 5], kmeans.cluster_centers_[:, 1],
            s=200, c='red', marker='X', label='Centroïdes')
plt.title('Répartition des observations par cluster (K=2)')
plt.xlabel('Glucose')
plt.ylabel('BMI')
plt.legend()
plt.show()

On utilise alors la PCA pour réduire le nombre de dimensions à deux, afin de mieux représenter les données.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(df_standard)

centroids_df = pd.DataFrame(kmeans.cluster_centers_, columns=df_standard.columns)
centroids_pca = pca.transform(centroids_df)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans.labels_, cmap='viridis')
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=200, c='red', marker='X', label='Centroïdes (PCA)')
plt.title('Visualisation des clusters après réduction PCA')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.legend()
plt.show()